<a href="https://colab.research.google.com/github/gibsonx/jlpt_simulator/blob/dev/graphs/n3/outliner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
if 'google.colab' in str(get_ipython()):
    !git clone https://github.com/gibsonx/jlpt_simulator.git
    %cd jlpt_simulator
    !git checkout dev
    
    !apt-get install python3-dev graphviz libgraphviz-dev pkg-config
    !pip install -r requirements.txt
else:
  print('Not running on CoLab')


Not running on CoLab


In [33]:
import json
import logging
import random
import time
from spire.doc import *
from spire.doc.common import *
import pandas as pd
import yaml
from tqdm import tqdm

# from langchain_community.embeddings import XinferenceEmbeddings
from libs.LLMs import *
import datetime
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

from graphs.n3.ExamTaskHandler import ExamTaskHandler
load_dotenv()

True

In [34]:
def collect_vocabulary(file_path):
    # Read the CSV file
    data = pd.read_csv(file_path)
    # Shuffle the rows and reset the index
    words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
    # Extract the second column (values) and convert to a single-line string
    vocab_string = ','.join(words.iloc[:, 1].astype(str).tolist())
    return vocab_string

In [35]:
# Import N3 Vocabulary
file_path = 'Vocab/n3.csv'
# Display the content of the CSV file
vocab_dict = collect_vocabulary(file_path)
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

In [21]:
vocab_dict

'こんやく,だいとうりょう,けっせき,もうふ,きたく,ぎょうぎ,さいちゅう,がく,しょうじる,どうか,がっかり,あずける,しゅうしょく,しんせき,がっき,じんこう,なわ,また,こっかい,さす,えんじょ,がい,パス,あわれ,ふくろ,たつ,ケース,じゅうし,あく,もとめる,ぶつり,せっけい,はかる,しゅふ,さいばん,すいみん,べつに,だい,ふそく,どういつ,てんこう,つゆ,なっとく,すがた,こす,きり,はく,かたる,とうちゃく,うたがう,たまたま,はん,のびる,てんけい,びみょう,じょうとう,しんけい,ちゅうもく,うれる,かわく,よほう,いいん,むすぶ,きみょう,せいかく,おくる,くれ,ユーモア,アイスクリーム,しはらい,ていこう,まぶしい,たから,けんかい,あいかわらず,えんき,すむ,さいわい,ろうどう,おろす,なくす,くむ,ある,ねんかん,そう,かんじょう,あらわす,けいき,あらわす,ほんにん,しゅつじょう,かかり,たにん,かんじ,ね,とじる,データ,このむ,とめる,いぜん,こぼれる,じき,はら,じょうだん,きしゃ,いわう,はじまり,いち,たたく,さく,かかえる,もんじ,ただ,つかむ,かる,けん,あたたかい,ゆうり,くだり,メンバー,スキー,だが,ろんぶん,かたづけ,てきせつ,さっきょく,がいしゅつ,きゅうか,めんせつ,いっか,なかば,むしあつい,つうじる,とら,こんかい,おん,いじ,ようき,しゅよう,だまる,てんねん,きづく,ちゅう,よこぎる,どうし,ぬの,ね,ルール,とも,とうじ,つきあい,じっと,かり,かける,しゅう,はさみ,かんじゃ,わずか,いきもの,え,でんごん,がん,さす,ちょくせつ,くわわる,おい,つらい,すごす,デモ,じけん,はなれる,たしょう,かんする,わる,のぞく,あたたまる,きこう,いじょう,けいい,ごうとう,つかれ,せいぜい,つまり,かいがい,しだい,げき,しゅぎ,あぶら,りえき,かわかす,しき,はなれる,テント,すませる,くむ,ひっし,きかん,あちこち,ふ,さんか,はで,すてき,アルバム,しんがく,へる,こじん,かよう,かく,ちえ,かんぜん,アウト,いずれ,ねがい,ねがう,スター,トンネル,かえす,かんしん,ろんそう,へんこう,げんかい,もの,ふじゆう,ことなる,よす,プラス,ぎもん,ひこう,ことによると,ここ,くさい,ちかごろ,とりあげる

#### load Models

In [22]:
from typing import List, Optional

from langchain_core.prompts import ChatPromptTemplate

from pydantic import BaseModel, Field

# instruction = """
# Section 1: Vocabulary and Grammar
# - Kanji reading (kanji_reading): 8 questions
# - Write Chinese characters (write_chinese): 6 questions
# - Word Meaning Selection (word_meaning): 11 questions
# - Synonyms substitution (synonym_substitution): 5 questions
# - word usage (word_usage): 5 questions
# - Grammar fill in the blank (sentence_grammar): 13 questions
# - Sentence sorting (sentence_sort): 5 questions
# - Grammar structure selection (sentence_structure): 4-5 questions
#
# Section 2: Reading Comprehension
# - Short passages (short_passage_read): 4 articles
# - Mid-size passages (midsize_passage_read): 2 articles
# - Long passages (long_passage_read): 1 articles
# - Information retrieval (info_retrieval): 1 articles
#
# Section 3: Listening Comprehension
# - Topic understanding (topic_understanding): 6 questions
# - Key understanding (keypoint_understanding): 6 questions
# - Summary understanding (summary_understanding): 3 questions
# - Active expression (active_expression): 4 questions
# - Immediate acknowledgment (immediate_ack): 9 questions
# """

instruction = """
Section 1: Vocabulary and Grammar
- のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい (kanji_reading): 8 questions
- このことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい (write_chinese): 6 questions

Section 2: Reading Comprehension
- Information retrieval (info_retrieval): 0 articles

Section 3: Listening Comprehension
- Topic understanding (topic_understanding): 0 questions

"""


direct_gen_outline_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            f"You are a japanese teacher. Your job is to write an outline for a JLPT Japanese-Language Proficiency Test N3 level exam paper. The words in outline must be presented in Japanese only. The complexity should be restricted to N3 level and respect japanese culture. The JLPT Japanese-Language Proficiency Test exam paper includes a mix of moderate and very difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"First, evenly pick Japanese words used in different part of sentence from 'Vocabulary' for questions in Vocabulary and Grammar, ensuring 40 percent of very difficult words. At Section 2 and 3, randomly choose topics in 'TopicList' for Reading Comprehension and Listening Comprehension Sections, don't repeat to choose a same word or topic"
            f"Second, you should abide by the provided exam instruction and decide the number of questions and content in the each Section."
            f"Finally, write the outline of the examination paper in japanese and provide question topics according to the instructions."
            f"Instruction: {instruction}",
        ),
        ("user", "TopicList: {topic_list}, Vocabulary: {vocab_dict}"),
    ]
)


## Data Strcuture

In [23]:
class QuestionTopic(BaseModel):
    topic: str = Field(..., title="a vocabulary or topic hint for a question")


class Subsection(BaseModel):
    subsection_title: str = Field(..., title="subsection English word in () only from the instruction")
    description: str = Field(..., title="giving the number of questions and requirements")
    question_topics: Optional[List[QuestionTopic]] = Field(
        default_factory=list
    )

    @property
    def as_str(self) -> str:
        question_topics_str = "\n".join(
            f"- **{qt.topic}**" for qt in self.question_topics
        )
        return f"### {self.subsection_title}\n\n{self.description}\n\n{question_topics_str}".strip()

class Section(BaseModel):
    section_title: str = Field(..., title="Title of the section")
    subsections: Optional[List[Subsection]] = Field(
        default_factory=list,
        title="Titles and reason for each subsection of the JLPT exam page.",
    )

    @property
    def as_str(self) -> str:
        subsections = "\n\n".join(
            subsection.as_str for subsection in self.subsections or []
        )
        return f"## {self.section_title}\n\n{subsections}".strip()


class Outline(BaseModel):
    page_title: str = Field(..., title="Title of the JLPT exam page")
    sections: List[Section] = Field(
        default_factory=list,
        title="Titles and descriptions for each section of the JLPT exam paper.",
    )

    @property
    def as_str(self) -> str:
        sections = "\n\n".join(section.as_str for section in self.sections)
        return f"# {self.page_title}\n\n{sections}".strip()


In [24]:
# Read the topics from a file, sort them, and print the sorted list
def process_topics(file_path):
    try:
        # Read the file
        with open(file_path, 'r', encoding='utf-8') as file:
            topics = file.readlines()

        # Remove any extra whitespace or newline characters
        topics = [topic.strip() for topic in topics if topic.strip()]

        # Shuffle the topics randomly
        random.shuffle(topics)

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

    except FileNotFoundError:
        print("The file was not found. Please check the file path.")
    except Exception as e:
        print("An error occurred:", str(e))

In [25]:
# Preload all topics from the file
with open("Vocab/topics.txt", "r", encoding="utf-8") as file:
    topics_list = [line.strip() for line in file]

generate_outline_direct = direct_gen_outline_prompt | azure_llm.with_structured_output(Outline)
initial_outline = generate_outline_direct.invoke({"topic_list": topics_list, "vocab_dict": vocab_dict})

2025-05-27 22:46:13,100 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"


In [26]:
from IPython.display import display, Markdown

display(Markdown(initial_outline.as_str))

# 日本語能力試験N3 模擬試験 問題概要

## 第1部：語彙・文法

### のことばの読み方 (kanji_reading)

次のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい。全8問。

- **さいちゅう**
- **がっかり**
- **しゅうしょく**
- **しょうじる**
- **びみょう**
- **じんこう**
- **ふそく**
- **きみょう**

### このことばを漢字で書く (write_chinese)

次のことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい。全6問。

- **むすぶ**
- **てんこう**
- **しゅふ**
- **しゅつじょう**
- **せいかく**
- **けっせき**

In [27]:
outliner_json = initial_outline.model_dump_json()
data = json.loads(outliner_json)  # Replace with your actual JSON data

start_time = time.time()

output_data = {'sections': []}

for section in data['sections']:
    output_section = {'section_title': section['section_title'], 'subsections': []}

    for subsection in tqdm(section['subsections'], desc=f"Processing {section['section_title']}"):
        function_name = subsection['subsection_title'].split(' (')[1].rstrip(')')
        questions = subsection['question_topics']

        for question in tqdm(questions, desc=f"Processing {subsection['subsection_title']}"):
            handler = ExamTaskHandler()
            func = getattr(handler, function_name, None)

            if func:
                max_attempts = 2
                original_topic = question['topic']  # Optional: Track original topic
                for attempt in range(max_attempts):
                    try:
                        result = func(question['topic'])
                        question['result'] = result
                        break  # Exit on success
                    except Exception as e:
                        if attempt < max_attempts - 1:
                            # Replace topic and retry
                            question['topic'] = random.choice(topics_list)
                        else:
                            question['result'] = f"Error after {max_attempts} attempts: {str(e)}"
            else:
                question['result'] = f"Method {function_name} not found"

        output_subsection = {
            'subsection_title': subsection['subsection_title'],
            'description': subsection['description'],
            'question_topics': questions
        }
        output_section['subsections'].append(output_subsection)

    output_data['sections'].append(output_section)

# End the timer
end_time = time.time()

# Calculate the total execution time
execution_time = end_time - start_time

print(f"Total execution time: {execution_time:.2f} seconds")

Processing のことばの読み方 (kanji_reading):   0%|          | 0/8 [00:00<?, ?it/s]

---WEB SEARCH---
---Generator----


2025-05-27 22:46:19,162 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 22:46:19,164 - INFO - Generated message: <li>
    試験の<u>最中</u>に電話が鳴りました。
    <ol>
        <li>さいじゅう</li>
        <li>さいちゅう</li>
        <li>さいしょう</li>
        <li>さいなか</li>
    </ol>
</li>


---REVISOR---


2025-05-27 22:46:39,805 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 22:46:39,808 - INFO - Refelect message: Thank you for your submission. Here is a detailed critique and recommendations based on the provided question:

1. Content Accuracy:
- The vocabulary focus on 最中 and its reading さいちゅう is appropriate for JLPT N3 level, as this is a commonly tested word.
- The sentence "試験の最中に電話が鳴りました。" is clear, natural, and contextually appropriate, reflecting a typical situation.
- The distractors (さいじゅう, さいしょう, さいなか) are plausible readings but only さいちゅう is correct here, which is good for testing recognition.

2. Question and Answer Quality:
- The question is straightforward: asking for the correct reading of the underlined kanji.
- The answer choices are clearly presented and unambiguous.
- The difficulty level fits JLPT N3, as recognizing the c

...

---REVISOR---


2025-05-27 22:51:31,352 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 22:51:31,354 - INFO - Refelect message: Thank you for your submission. Here is my detailed critique and recommendations:

1. Content Accuracy:  
- The vocabulary question correctly targets the word "けっせき" in the context of a meeting, which is appropriate for JLPT N3 level.  
- The correct answer is "欠席," which means "absence." The other options are plausible kanji compounds but incorrect in this context, making this a good multiple-choice question to test kanji knowledge and vocabulary.  
- The sentence is clear and relevant, and the vocabulary fits the N3 level well.

2. Question and Answer Quality:  
- The question is straightforward and unambiguous. The sentence context clearly indicates the meaning of "けっせき" as "absence."  
- The distractors are well chosen:  
  - 結席

--- AI Reviser feels Good Enough ---
--- Formatter ---


2025-05-27 22:51:32,160 - INFO - HTTP Request: POST https://ai-rolandaws880125ai409947751408.openai.azure.com/openai/deployments/gpt-4.1-mini/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"
2025-05-27 22:51:32,162 - INFO - Formatted message: {'html_question': '彼は会議を<u>けっせき</u>しました。', 'choices': ['結席', '欠石', '欠席', '結石'], 'correct_answer': 3}

Processing 第1部：語彙・文法: 100%|██████████| 2/2 [05:19<00:00, 159.51s/it]

Total execution time: 319.03 seconds


In [28]:
yaml_str = yaml.dump(output_data, allow_unicode=True, sort_keys=False, default_flow_style=False)
# Display as formatted YAML
display(Markdown(yaml_str))

sections:
- section_title: 第1部：語彙・文法
  subsections:
  - subsection_title: のことばの読み方 (kanji_reading)
    description: 次のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい。全8問。
    question_topics:
    - topic: さいちゅう
      result:
        html_question: 試験の<u>最中</u>に電話が鳴りました。
        correct_answer: 2
        choices:
        - さいじゅう
        - さいちゅう
        - さいしょう
        - さいなか
    - topic: がっかり
      result:
        html_question: 旅行が中止になって<u>_____</u>した。
        correct_answer: 2
        choices:
        - しっかり
        - がっかり
        - きっかり
        - ばっかり
    - topic: しゅうしょく
      result:
        html_question: 日本語の文法では<u>修飾</u>の役割が大切です。
        correct_answer: 1
        choices:
        - しゅうしょく
        - しゅうそく
        - しゅしょう
        - しゅしょうく
    - topic: しょうじる
      result:
        html_question: 新しい問題が<u>生じる</u>ことがあります。
        correct_answer: 2
        choices:
        - うまじる
        - しょうじる
        - せいじる
        - いきじる
    - topic: びみょう
      result:
        html_question: その映画の評価は<u>微妙</u>だった。
        correct_answer: 2
        choices:
        - みみょう
        - びみょう
        - びみょ
        - みみょ
    - topic: 地域社会への貢献について話す
      result:
        html_question: 彼は地域の<u>貢献</u>について発表した。
        correct_answer: 1
        choices:
        - こうけん
        - こうこん
        - こうけい
        - こうこんえん
    - topic: ふそく
      result:
        html_question: 最近、この町では水の<u>不足</u>が問題になっています。
        correct_answer: 1
        choices:
        - ふそく
        - ぶそく
        - ふしょく
        - ぶしょく
    - topic: きみょう
      result:
        html_question: 彼は<u>奇妙</u>な話をしていた。
        correct_answer: 2
        choices:
        - きみつ
        - きみょう
        - きみょお
        - きびょう
  - subsection_title: このことばを漢字で書く (write_chinese)
    description: 次のことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい。全6問。
    question_topics:
    - topic: むすぶ
      result:
        html_question: このひもをしっかり<u>むすぶ</u>必要があります。
        correct_answer: 1
        choices:
        - 結ぶ
        - 編む
        - 織る
        - 組む
    - topic: てんこう
      result:
        html_question: 明日の<u>てんこう</u>についてニュースで確認しました。
        choices:
        - 天校
        - 天候
        - 天効
        - 天幸
        correct_answer: 2
    - topic: しゅふ
      result:
        html_question: 母は<u>しゅふ</u>として毎日家事をしています。
        correct_answer: 1
        choices:
        - 主婦
        - 主夫
        - 主付
        - 主婦人
    - topic: しゅつじょう
      result:
        html_question: 彼は全国大会に<u>しゅつじょう</u>することが決まりました。
        correct_answer: 1
        choices:
        - 出場
        - 出情
        - 出条
        - 出陽
    - topic: せいかく
      result:
        html_question: 彼はとても<u>せいかく</u>が明るい人です。
        correct_answer: 1
        choices:
        - 性格
        - 正格
        - 精格
        - 生格
    - topic: けっせき
      result:
        html_question: 彼は会議を<u>けっせき</u>しました。
        choices:
        - 結席
        - 欠石
        - 欠席
        - 結石
        correct_answer: 3


In [29]:
output_data

{'sections': [{'section_title': '第1部：語彙・文法',
   'subsections': [{'subsection_title': 'のことばの読み方 (kanji_reading)',
     'description': '次のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい。全8問。',
     'question_topics': [{'topic': 'さいちゅう',
       'result': {'html_question': '試験の<u>最中</u>に電話が鳴りました。',
        'correct_answer': 2,
        'choices': ['さいじゅう', 'さいちゅう', 'さいしょう', 'さいなか']}},
      {'topic': 'がっかり',
       'result': {'html_question': '旅行が中止になって<u>_____</u>した。',
        'correct_answer': 2,
        'choices': ['しっかり', 'がっかり', 'きっかり', 'ばっかり']}},
      {'topic': 'しゅうしょく',
       'result': {'html_question': '日本語の文法では<u>修飾</u>の役割が大切です。',
        'correct_answer': 1,
        'choices': ['しゅうしょく', 'しゅうそく', 'しゅしょう', 'しゅしょうく']}},
      {'topic': 'しょうじる',
       'result': {'html_question': '新しい問題が<u>生じる</u>ことがあります。',
        'correct_answer': 2,
        'choices': ['うまじる', 'しょうじる', 'せいじる', 'いきじる']}},
      {'topic': 'びみょう',
       'result': {'html_question': 'その映画の評価は<u>微妙</u>だった。',
        'correct_ans

In [30]:
def render_to_html(sections):
    html = '<html><body style="text-align: left;">\n'
    for section in sections:
        html += f'<h1>{section.get("section_title", "")}</h1>\n'
        for subsection in section.get("subsections", []):
            html += f'<h2>{subsection.get("subsection_title", "")}</h2>\n'
            if "description" in subsection:
                html += f'<p>{subsection["description"]}</p>\n'
            for idx, qtopic in enumerate(subsection.get("question_topics", []),1):
                # html += f'<h3>{qtopic.get("topic", "")}</h3>\n'
                result = qtopic.get("result", {})
                if isinstance(result, dict):
                    if "html_article" in result:
                        html += result["html_article"] + "\n"
                    if "html_question" in result:
                        html += f'<p><strong>{idx} . {result["html_question"]}</strong></p>\n'
                        if "choices" in result:
                            correct = result.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(result["choices"], 1):
                                if idx == correct:
                                    html += f"<li>{idx}. <b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{idx}. {choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in result:
                            html += f'<p><em>Explanation: {result["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], list):
                        for q in result["questions"]:
                            html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                            if "choices" in q:
                                correct = q.get("correct_answer", -1)
                                html += '<ul>\n'
                                for idx, choice in enumerate(q["choices"], 1):
                                    if idx == correct:
                                        html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                    else:
                                        html += f"<li>{choice}</li>\n"
                                html += '</ul>\n'
                            if "explanation" in q:
                                html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
                    if "questions" in result and isinstance(result["questions"], dict):
                        q = result["questions"]
                        html += f'<p><strong>Q: {q["question"]}</strong></p>\n'
                        if "choices" in q:
                            correct = q.get("correct_answer", -1)
                            html += '<ul>\n'
                            for idx, choice in enumerate(q["choices"], 1):
                                if idx == correct:
                                    html += f"<li><b>{choice}</b> <span style='color:green;'>(correct)</span></li>\n"
                                else:
                                    html += f"<li>{choice}</li>\n"
                            html += '</ul>\n'
                        if "explanation" in q:
                            html += f'<p><em>Explanation: {q["explanation"]}</em></p>\n'
    html += '</body></html>'
    return html

In [38]:
html_output = render_to_html(output_data['sections'])
display(Markdown(html_output))

<html><body style="text-align: left;">
<h1>第1部：語彙・文法</h1>
<h2>のことばの読み方 (kanji_reading)</h2>
<p>次のことばの読み方として最もよいものを、1・2・3・4から一つえらびなさい。全8問。</p>
<p><strong>1 . 試験の<u>最中</u>に電話が鳴りました。</strong></p>
<ul>
<li>1. さいじゅう</li>
<li>2. <b>さいちゅう</b> <span style='color:green;'>(correct)</span></li>
<li>3. さいしょう</li>
<li>4. さいなか</li>
</ul>
<p><strong>2 . 旅行が中止になって<u>_____</u>した。</strong></p>
<ul>
<li>1. しっかり</li>
<li>2. <b>がっかり</b> <span style='color:green;'>(correct)</span></li>
<li>3. きっかり</li>
<li>4. ばっかり</li>
</ul>
<p><strong>3 . 日本語の文法では<u>修飾</u>の役割が大切です。</strong></p>
<ul>
<li>1. <b>しゅうしょく</b> <span style='color:green;'>(correct)</span></li>
<li>2. しゅうそく</li>
<li>3. しゅしょう</li>
<li>4. しゅしょうく</li>
</ul>
<p><strong>4 . 新しい問題が<u>生じる</u>ことがあります。</strong></p>
<ul>
<li>1. うまじる</li>
<li>2. <b>しょうじる</b> <span style='color:green;'>(correct)</span></li>
<li>3. せいじる</li>
<li>4. いきじる</li>
</ul>
<p><strong>5 . その映画の評価は<u>微妙</u>だった。</strong></p>
<ul>
<li>1. みみょう</li>
<li>2. <b>びみょう</b> <span style='color:green;'>(correct)</span></li>
<li>3. びみょ</li>
<li>4. みみょ</li>
</ul>
<p><strong>6 . 彼は地域の<u>貢献</u>について発表した。</strong></p>
<ul>
<li>1. <b>こうけん</b> <span style='color:green;'>(correct)</span></li>
<li>2. こうこん</li>
<li>3. こうけい</li>
<li>4. こうこんえん</li>
</ul>
<p><strong>7 . 最近、この町では水の<u>不足</u>が問題になっています。</strong></p>
<ul>
<li>1. <b>ふそく</b> <span style='color:green;'>(correct)</span></li>
<li>2. ぶそく</li>
<li>3. ふしょく</li>
<li>4. ぶしょく</li>
</ul>
<p><strong>8 . 彼は<u>奇妙</u>な話をしていた。</strong></p>
<ul>
<li>1. きみつ</li>
<li>2. <b>きみょう</b> <span style='color:green;'>(correct)</span></li>
<li>3. きみょお</li>
<li>4. きびょう</li>
</ul>
<h2>このことばを漢字で書く (write_chinese)</h2>
<p>次のことばを漢字で書くとき、最もよいものを、１・２・３・４から一つえらびなさい。全6問。</p>
<p><strong>1 . このひもをしっかり<u>むすぶ</u>必要があります。</strong></p>
<ul>
<li>1. <b>結ぶ</b> <span style='color:green;'>(correct)</span></li>
<li>2. 編む</li>
<li>3. 織る</li>
<li>4. 組む</li>
</ul>
<p><strong>2 . 明日の<u>てんこう</u>についてニュースで確認しました。</strong></p>
<ul>
<li>1. 天校</li>
<li>2. <b>天候</b> <span style='color:green;'>(correct)</span></li>
<li>3. 天効</li>
<li>4. 天幸</li>
</ul>
<p><strong>3 . 母は<u>しゅふ</u>として毎日家事をしています。</strong></p>
<ul>
<li>1. <b>主婦</b> <span style='color:green;'>(correct)</span></li>
<li>2. 主夫</li>
<li>3. 主付</li>
<li>4. 主婦人</li>
</ul>
<p><strong>4 . 彼は全国大会に<u>しゅつじょう</u>することが決まりました。</strong></p>
<ul>
<li>1. <b>出場</b> <span style='color:green;'>(correct)</span></li>
<li>2. 出情</li>
<li>3. 出条</li>
<li>4. 出陽</li>
</ul>
<p><strong>5 . 彼はとても<u>せいかく</u>が明るい人です。</strong></p>
<ul>
<li>1. <b>性格</b> <span style='color:green;'>(correct)</span></li>
<li>2. 正格</li>
<li>3. 精格</li>
<li>4. 生格</li>
</ul>
<p><strong>6 . 彼は会議を<u>けっせき</u>しました。</strong></p>
<ul>
<li>1. 結席</li>
<li>2. 欠石</li>
<li>3. <b>欠席</b> <span style='color:green;'>(correct)</span></li>
<li>4. 結石</li>
</ul>
</body></html>

In [39]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
with open(f"output/JLPT_{timestamp}.html", "w", encoding="utf-8") as file:
    file.write(html_output)

In [37]:
# Create a Document object
document = Document()
# Add a section to the document
section = document.AddSection()
# Set the page margins to 72 points (72 points = 1 inch)
section.PageSetup.Margins.All = 72

# Add a paragraph to the section
paragraph = section.AddParagraph()

# Add the HTML string to the paragraph
paragraph.AppendHTML(html_output)

# Save the result document to a DOCX file
document.SaveToFile(f"output/JLPT_{timestamp}.docx", FileFormat.Docx2016)
# Or save the result document to a DOC file
# document.SaveToFile("HtmlStringToDoc.doc", FileFormat.Doc)

document.Close()